In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.datasets import load_files
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import KeyedVectors
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences 
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SimpleRNN
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.utils.np_utils import to_categorical
from keras.models import Model
from keras.engine import Input

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
#Reading the data
data = pd.read_csv('/content/drive/MyDrive/Classroom/NLP/competition_1/malayalam_news_train.csv')

In [5]:
data.head()

,headings,label
0,കൊച്ചി ചെരുപ്പ് വിതരണ കേന്ദ്രത്തിലെ തീപിടിത്തം...,business
1,ഈ​​​സ്റ്റേ​​​ണ്‍ ക​​​റി പൗ​​​ഡ​​​റി​​​ന് പു​​​...,business
2,ശ്രീശാന്തിന്റെ ആജീവനാന്ത വിലക്ക് പിന്‍വലിച്ചു,sports
3,റയല്‍ മാഡ്രിഡിനെ ഞെട്ടിച്ച യുവതാരം ബ്രസീല്‍ ടീ...,sports
4,"എനിക്ക് ശ്രീനിവാസന്‍ ആകേണ്ട, ലോഹിതദാസ് മതി : ക...",entertainment


In [6]:
print(data['headings'])

0       കൊച്ചി ചെരുപ്പ് വിതരണ കേന്ദ്രത്തിലെ തീപിടിത്തം...
1       ഈ​​​സ്റ്റേ​​​ണ്‍ ക​​​റി പൗ​​​ഡ​​​റി​​​ന് പു​​​...
2           ശ്രീശാന്തിന്റെ ആജീവനാന്ത വിലക്ക് പിന്‍വലിച്ചു
3       റയല്‍ മാഡ്രിഡിനെ ഞെട്ടിച്ച യുവതാരം ബ്രസീല്‍ ടീ...
4       എനിക്ക് ശ്രീനിവാസന്‍ ആകേണ്ട, ലോഹിതദാസ് മതി : ക...
                              ...                        
5031    മുല്ലപ്പൂ വിപ്ലവത്തിലെ പുതിയ പോസ്റ്റര്‍ പുറത്ത...
5032    ബാ​ങ്ക് അ​ക്കൗ​ണ്ടി​ന് ആ​ധാ​ര്‍ കാ​ര്‍​ഡ്: ഹൈ​...
5033    തോല്‍വിയുടെ ക്ഷീണത്തില്‍ നിന്ന് ടീമിന് കരകയറണം...
5034    അഹങ്കാരിയും ജാഡക്കാരനുമൊന്നുമല്ല, മമ്മൂസ് വെറു...
5035    ലൂസിഫറില്‍ കട്ട കലിപ്പില്‍ കലാഭവന്‍ ഷാജോണ്‍! ല...
Name: headings, Length: 5036, dtype: object


Removing punctuations

In [7]:
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

In [8]:
for ele in data:
  if ele in punc:
    data = data.replace(ele, "")

In [9]:
#Tokenization
max_features = 1200 #number of words to keep. 
#1200 is the number of unique words in the corpus.
tokenizer = Tokenizer(nb_words=max_features, split=' ')
tokenizer.fit_on_texts(data['headings'].values)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/text.py:180: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [10]:
#Unique words and and their count
tokenizer.word_counts

OrderedDict([('കൊച്ചി', 6),
             ('ചെരുപ്പ്', 1),
             ('വിതരണ', 5),
             ('കേന്ദ്രത്തിലെ', 1),
             ('തീപിടിത്തം', 1),
             ('അന്വേഷണം', 6),
             ('തുടങ്ങി', 41),
             ('അട്ടിമറി', 3),
             ('സാധ്യതയും', 1),
             ('പരിശോധിക്കുന്നു', 1),
             ('ഈ\u200b\u200b\u200bസ്റ്റേ\u200b\u200b\u200bണ്\u200d', 1),
             ('ക\u200b\u200b\u200bറി', 1),
             ('പൗ\u200b\u200b\u200bഡ\u200b\u200b\u200bറി\u200b\u200b\u200bന്',
              1),
             ('പു\u200b\u200b\u200bര\u200b\u200b\u200bസ്കാ\u200b\u200b\u200bരം',
              1),
             ('സം\u200b\u200b\u200bസ്ഥാ\u200b\u200b\u200bന', 1),
             ('മ\u200b\u200b\u200bലി\u200b\u200b\u200bനീ\u200b\u200b\u200bക\u200b\u200b\u200bര\u200b\u200b\u200bണ',
              1),
             ('നി\u200b\u200b\u200bയ\u200b\u200b\u200bന്ത്ര\u200b\u200b\u200bണ',
              1),
             ('ബോ\u200b\u200b\u200bര്\u200d\u200b\u200b\u200bഡി\u200b\u200b\u200

In [11]:
X = tokenizer.texts_to_sequences(data['headings'].values)
X = pad_sequences(X, padding = 'post') #Zero padding at the end of the sequence

In [12]:
labels = data['label'].tolist()
le = preprocessing.LabelEncoder()
Y = le.fit_transform(labels)

In [13]:
Y = to_categorical(Y)
print(Y)

[[1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [14]:
from sklearn.model_selection import KFold

In [15]:
kf = KFold(n_splits=4)

In [16]:
for training, testing in kf.split(X):
  print("TRAIN:", X[training], "TESTING:",X[testing])
  X_train, X_test = X[training], X[testing]
  Y_train, Y_test = Y[training], Y[testing]

TRAIN: [[826 857 228 ...   0   0   0]
 [  2 828 472 ...   0   0   0]
 [541 108  96 ...   0   0   0]
 ...
 [ 28 603 654 ...   0   0   0]
 [617   0   0 ...   0   0   0]
 [435 902  11 ...   0   0   0]] TESTING: [[1022 1023   79 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [ 286  176  126 ...    0    0    0]
 ...
 [1137   77   76 ...    0    0    0]
 [  35  679  919 ...    0    0    0]
 [  36  279   69 ...    0    0    0]]
TRAIN: [[1022 1023   79 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [ 286  176  126 ...    0    0    0]
 ...
 [  28  603  654 ...    0    0    0]
 [ 617    0    0 ...    0    0    0]
 [ 435  902   11 ...    0    0    0]] TESTING: [[ 826  857  228 ...    0    0    0]
 [   2  828  472 ...    0    0    0]
 [ 541  108   96 ...    0    0    0]
 ...
 [1168  399  991 ...    0    0    0]
 [1153  314  491 ...    0    0    0]
 [1182  317   19 ...    0    0    0]]
TRAIN: [[1022 1023   79 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [ 286  176

In [17]:
#Printing the size of the train data, train label, test data and test label
print("Shape train data = ",np.shape(X_train))
print("Shape of train label = ",np.shape(Y_train))
print("Shape of test data = ",np.shape(X_test))
print("Shape of test label = ",np.shape(Y_test))

Shape train data =  (3777, 24)
Shape of train label =  (3777, 3)
Shape of test data =  (1259, 24)
Shape of test label =  (1259, 3)


In [18]:
embed_dim = 500
hidden_layer = 100
model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(Dropout(0.2))
model.add(SimpleRNN(hidden_layer))
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 24, 500)           600000    
_________________________________________________________________
dropout (Dropout)            (None, 24, 500)           0         
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 100)               60100     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 153       
Total params: 675,403
Trainable params: 675,403
Non-trainable params: 0
__________________________________________________

In [19]:
model.fit(X_train,Y_train,epochs = 20, batch_size = 32)

Epoch 1/20
119/119 [==============================] - 3s 28ms/step - loss: 0.6884 - accuracy: 0.6921
Epoch 2/20
119/119 [==============================] - 3s 28ms/step - loss: 0.3595 - accuracy: 0.8660
Epoch 3/20
119/119 [==============================] - 3s 28ms/step - loss: 0.2039 - accuracy: 0.9200
Epoch 4/20
119/119 [==============================] - 3s 27ms/step - loss: 0.1787 - accuracy: 0.9349
Epoch 5/20
119/119 [==============================] - 3s 28ms/step - loss: 0.1487 - accuracy: 0.9404
Epoch 6/20
119/119 [==============================] - 3s 27ms/step - loss: 0.1368 - accuracy: 0.9455
Epoch 7/20
119/119 [==============================] - 3s 28ms/step - loss: 0.1217 - accuracy: 0.9497
Epoch 8/20
119/119 [==============================] - 3s 27ms/step - loss: 0.1339 - accuracy: 0.9447
Epoch 9/20
119/119 [==============================] - 3s 27ms/step - loss: 0.1239 - accuracy: 0.9513
Epoch 10/20
119/119 [==============================] - 3s 28ms/step - loss: 0.1239 - accura

In [20]:
score = model.evaluate(X_train, Y_train, verbose = 1, batch_size = 32)
print("Accuracy: %.2f" % (score[1]*100))

119/119 [==============================] - 1s 5ms/step - loss: 0.0881 - accuracy: 0.9616
Accuracy: 96.16


In [21]:
score = model.evaluate(X_test, Y_test, verbose = 1, batch_size = 32)
print("Accuracy: %.2f" % (score[1]*100))

40/40 [==============================] - 0s 5ms/step - loss: 0.6409 - accuracy: 0.8483
Accuracy: 84.83


In [22]:
test = pd.read_csv('/content/drive/MyDrive/Classroom/NLP/competition_1/malayalam_news_test.csv')
test.head()

,ID,headings
0,MAL_0001,അന്താരാഷ്ട്ര ഇന്റര്‍നെറ്റ് സേവനം താല്‍ക്കാലികമ...
1,MAL_0002,"മഞ്ഞ് ചതിച്ചാശാനേ!!!!, ഹാന്നോവറിന്റെ പ്രതീക്ഷക..."
2,MAL_0003,"12 സീസണ്‍ എട്ടു ചാമ്ബ്യന്മാര്‍, പ്രവചിക്കാന്‍ ..."
3,MAL_0004,\nമുംബൈ സിറ്റിയോടു തോറ്റിട്ടും യോഗ്യത നേടി ഗോവ...
4,MAL_0005,മദ്യപിച്ചെത്തിയ നടന്മാര്‍ തമ്മില്‍ കയ്യാങ്കളി;...


In [23]:
print(test['headings'])

0       അന്താരാഷ്ട്ര ഇന്റര്‍നെറ്റ് സേവനം താല്‍ക്കാലികമ...
1       മഞ്ഞ് ചതിച്ചാശാനേ!!!!, ഹാന്നോവറിന്റെ പ്രതീക്ഷക...
2       12 സീസണ്‍ എട്ടു ചാമ്ബ്യന്മാര്‍, പ്രവചിക്കാന്‍ ...
3       \nമുംബൈ സിറ്റിയോടു തോറ്റിട്ടും യോഗ്യത നേടി ഗോവ...
4       മദ്യപിച്ചെത്തിയ നടന്മാര്‍ തമ്മില്‍ കയ്യാങ്കളി;...
                              ...                        
1255    സാക്ഷാല്‍ വിരാട് കോഹ്ലിയെ പോലും ഞെട്ടിച്ച്‌ ജസ...
1256    ചാമ്ബ്യന്‍സ് ലീഗ്; ബയേണിനെ തകര്‍ത്ത് ലിവര്‍പൂള...
1257    20 രൂപയുടെ നാണയമിറക്കാന്‍ കേന്ദ്ര ധനകാര്യ മന്ത...
1258    ഇന്ത്യന്‍ സമ്ബദ് വ്യവസ്ഥ 7.3 ശതമാനം വളര്‍ച്ച ന...
1259    വൈ ദിസ് കൊലച്ചതി? ഒറ്റ രാത്രികൊണ്ട് ലക്ഷക്കണക്...
Name: headings, Length: 1260, dtype: object


In [24]:
test = tokenizer.texts_to_sequences(test['headings'].values)
test = pad_sequences(test, padding = 'post')

In [25]:
class_label = model.predict_classes(test)
print(le.inverse_transform(class_label))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
['business' 'sports' 'entertainment' ... 'sports' 'sports' 'sports']


In [26]:
pred_labels=le.inverse_transform(class_label)
print(pred_labels)

['business' 'sports' 'entertainment' ... 'sports' 'sports' 'sports']


In [27]:
df = pd.read_csv('/content/drive/MyDrive/Classroom/NLP/competition_1/ROLLNO_Task_1_submission.csv')
df.head()

,ID,Predicted labels
0,MAL_0001,NaN
1,MAL_0002,NaN
2,MAL_0003,NaN
3,MAL_0004,NaN
4,MAL_0005,NaN


In [28]:
df['Predicted labels']=pred_labels

In [29]:
pred_labels

array(['business', 'sports', 'entertainment', ..., 'sports', 'sports',
       'sports'], dtype='<U13')

In [30]:
df.to_csv(r'/content/drive/MyDrive/Classroom/NLP/competition_1/pred_5.csv', index=False)